In [11]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
from graphviz import Digraph
import random
from IPython.display import IFrame

In [12]:
seed = 53
np.random.seed(seed)
torch.manual_seed(seed)

In [30]:
class node(object):
    no =0
#     node_type = 'simple'
    def __init__(self, input_shape = (0, 0, 0), output_shape = (0, 0, 0)): # c, i1, i2
        node.no += 1
        self.no = node.no
        self.in_adj = []
        self.out_adj = []
        self.input_shape = input_shape
        self.output_shape = output_shape
    
#     def set_shape(self, input_shape, output_shape):
#         self.input_shape = input_shape
#         self.output_shape = output_shape
        

    def describe_adj_list(self, in_adj, out_adj):
        self.in_adj = in_adj
        self.out_adj = out_adj

    def out_shape(self):
        pass
    
    def remove(self):
        ## needed in graph class
        pass
    
    
    def __str__(self):
        return str(self.no) + " " + str(type(self)) 

    def __repr__(self):
        return self.__str__()

In [37]:
class convolution_block(nn.Module, node):
    def __init__(self, in_channels, in_h, in_w, out_channels, kernel_size, padding = 0, stride = 1):
        super(convolution_block, self).__init__()
        node.__init__(self, (in_channels, in_h, in_w))
        self.in_channels  = in_channels
        self.out_channels = out_channels 
        self.kernel_size = kernel_size
        self.stride = stride 
        self.padding = padding
        self.output_shape = self.out_shape()
        
        # NN Layers
        self.conv_layer = nn.Conv2d(self.in_channels, self.out_channels, self.kernel_size, self.stride, self.padding)
        self.batch_norm = nn.BatchNorm2d(self.out_channels)
        self.relu = nn.ReLU()
    
    def refresh(self, in_channels, in_h, in_w, out_channels, kernel_size, padding = 0, stride=1):
        self.in_channels = in_channels
        self.out_channels = out_channels 
        self.kernel_size  = kernel_size 
        self.stride = stride 
        self.padding = padding
        self.input_shape = (in_channels, in_h, in_w)
        self.output_shape = self.out_shape()
        
        ## NN Layers
        self.conv_layer = nn.Conv2d(self.in_channels, self.out_channels, self.kernel_size, self.stride, self.padding)
        self.batch_norm = nn.BatchNorm2d(self.out_channels)
        self.relu = nn.ReLU()
                
    def out_shape(self):
        c, h, w = self.input_shape
        C = self.out_channels
        H = (h + 2*self.padding - self.kernel_size)/self.stride + 1
        W = (w + 2*self.padding - self.kernel_size)/self.stride + 1
        return (C, H, W)
    
    def forward(self, x):
        x = self.conv_layer(x)
        x = self.batch_norm(x)
        x = self.relu(x)
        return x 

In [38]:
class max_pool_block(nn.Module, node):  
    def __init__(self, in_channels, in_h, in_w, kernel_size, padding = 0, stride = 1):
        super(max_pool_node, self).__init__()    
        node.__init__(self, (in_channels, in_h, in_w))
        self.kernel_size = kernel_size
        self.stride = stride 
        self.padding = padding
        self.output_shape = self.out_shape()
        
        ## NN Layer
        self.max_pool_layer = nn.MaxPool2d(self.kernel_size, self.stride, self.padding)
        
    def forward(self, x):
        return self.max_pool_layer(x)
    
    def out_shape(self):
        c, h, w = self.input_shape
        C = c
        H = (h + 2*self.padding - self.kernel_size)/self.stride + 1
        W = (w + 2*self.padding - self.kernel_size)/self.stride + 1
        return (C, H, W)

In [39]:
class merge_block(node):
    def __init__(self, parents, child):
        super(merge_block, self).__init__()
        self.describe_adj_list(parents, child)
        
class add_block(nn.Module, merge_block): 
    def __init__(self, parents, child):
        super(add_block, self).__init__()
        merge_block.__init__(self, parents, child)
        self.input_shape = self.in_adj[0].output_shape
        self.output_shape = self.out_shape()
                
    def forward(self, x, y):
        return x+y 
        
    def out_shape(self):
        return self.input_shape

class concat_node(merge_block):
    # Define Later
    pass

class convex_merge_node(merge_block):
    # Define Later
    pass

In [50]:
# # Make it a part of Net
# class fullyConnectedBlock(nn.Module,node) :
#     def __init__(self, net) : 
#         super(fullyConnected,self).__init__()
#         node.__init__(self)
#         self.net = net
#         sh = net.int_to_node[net.topsort[-1]].output_shape
#         out = sh[0]*sh[1]*sh[2]
#         self.fc1 = nn.Linear(out,16)
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(16,10)
        
#     def forward(self,x) :
#         out = self.net(x)
#         out  =  out.view(out.shape[0], -1)
# #         print 'out.shape', out.shape
# #         [1 x 75264], m2: [2352 x 16]
#         out = self.fc1(out)
#         out = self.relu(out)
#         out = self.fc2(out)
#         return out

# BATCH = 4
# BEGIN_IN_CHANNELS = 3 
# def addLinearLayers(net):
# #     t = [32,3,28,28]
#     net = fullyConnected(net)
#     return net


In [53]:
class FlattenBlock(nn.Module, node):
    def __init__(self, input_shape):
        super(FlattenBlock, self).__init__()
        node.__init__(self, input_shape, (input_shape[0]*input_shape[1]*input_shape[2],))
    def forward(self, x):
        return x.view(x.shape[0], -1)

In [57]:
# Make it a part of Net
class fullyConnectedBlock(nn.Module,node) :
    def __init__(self, input_units, output_units, activation = False) : 
        super(fullyConnectedBlock,self).__init__()
        node.__init__(self, (input_units, ), (output_units, ))
        self.layers = []
        self.layers.append(nn.Linear(input_units, output_units))
        if activation:
            self.layers.append(nn.ReLU())
     
    def forward(self, x) :
        out = x
        for layer in self.layers:
            out = layer(out)
        return out

# BATCH = 4
# BEGIN_IN_CHANNELS = 3 
# def addLinearLayers(net):
# #     t = [32,3,28,28]
#     net = fullyConnected(net)
#     return net


In [101]:
class Network(nn.Module, object):
    def __init__(self):
        super(Network, self).__init__()
        self.adj_mat = {}
        self.adj_list = {}
        self.int_to_node = {}
        self.node_to_int = {}
        self.nodes = []
        self.conv_blocks = []
        self.max_pool_blocks = [] 
        self.topsort = []
        self.rank_in_topsort = {}
        self.max_no = 0
        self.node_dict = {}
        #self.fc = nn.Linear(3 * 28 * 28, 10)
        
    def __init__(self, adj_list, int_to_node):
        super(Network, self).__init__()        
#         assert len(adj_list) == assert(int_to_node)
        self.adj_list = adj_list
        self.adj_mat = self.get_adj_mat(self.adj_list)
        self.nodes = int_to_node.keys()
        self.int_to_node = int_to_node
        self.node_dict = {}
        self.node_to_int = self.get_node_to_int(self.int_to_node)
        self.max_no = max(self.int_to_node)
        self.conv_blocks, self.max_pool_blocks = self.get_conv_and_max_pool_blocks()
        self.topsort = []
        self.rank_in_topsort = {}
        self.topsorting()
        # hardcoded part
        #self.fc = torch.nn.Linear(3 * 28 * 28, 10)
        
    def get_adj_mat(self, adj_list):
        adj_mat = {}
        nodes = adj_list.keys()
        for x in nodes:
            adj_mat[x] = {}
            for y in nodes:
                adj_mat[x][y] = 0
        for cnode, li in adj_list.items():
            for par in li[0]:
                adj_mat[par][cnode] = 1
            for child in li[1]:
                adj_mat[cnode][child] = 1
        return adj_mat
    
    def get_node_to_int(self, int_to_node):
        node_to_int = {}
        for no, cnode in int_to_node.items():
            node_to_int[cnode] = no
        return node_to_int

    def topsorting(self):
        # level problem
        topsort = []
        import Queue
        in_deg = {}
        q = Queue.Queue()
        for node in self.nodes:
            val  = len(self.adj_list[node][0])
            if val == 0:
                q.put(node)
            in_deg[node] = val
            
        while not q.empty():
            curr_node = q.get()
            topsort.append(curr_node)
            for child in self.adj_list[curr_node][1]:
                in_deg[child] -= 1
                if in_deg[child] == 0:
                    q.put(child)
        self.topsort = topsort
        self.set_rank_in_topsort()
    
    def set_rank_in_topsort(self):
        for ind, node_no in enumerate(self.topsort):
            self.rank_in_topsort[node_no]  = ind
        
    def createModel(self):
         self.node_dict = torch.nn.ModuleDict(self.node_dict)
        
    def forward(self, x):
        self.topsorting() # though this is not required here
        outputs = {}
        outputs[self.topsort[0]] = self.int_to_node[self.topsort[0]].forward(x)
        for ind in range(1, len(self.topsort)):
            node_no = self.topsort[ind]
            curr_node = self.int_to_node[node_no]
            outputs[node_no] = curr_node.forward(*map(lambda x: outputs[x], self.adj_list[node_no][0]))
        return outputs[self.topsort[-1]]
    
    def get_conv_and_max_pool_blocks(self):
        conv_blocks = []
        max_pool_blocks = []
        for x in self.nodes:
            if isinstance(self.int_to_node[x], convolution_block):
                conv_blocks.append(x)
                self.node_dict[str(x)]=self.int_to_node[x]
            elif isinstance(self.int_to_node[x], max_pool_block):
                max_pool_blocks.append(x)
                self.node_dict[str(x)]=self.int_to_node[x]
        return (conv_blocks, max_pool_blocks)
        
 
    def add_nodes_to_network(self, nodes):
        for curr_node in nodes:
            if curr_node not in self.node_to_int:
                self.max_no += 1
                self.adj_mat[self.max_no] = {}
                self.adj_list[self.max_no] = [[], []]
                self.node_to_int[curr_node] = self.max_no
                self.int_to_node[self.max_no] = curr_node
                self.nodes.append(self.max_no)
                if isinstance(curr_node, convolution_block):
                    self.conv_blocks.append(self.max_no)
                    self.node_dict[str(self.max_no)]=curr_node  
                elif isinstance(curr_node, max_pool_block):
                    self.max_pool_blocks.append(self.max_no)
                    self.node_dict[str(self.max_no)]=curr_node
                    
        for curr_node in nodes:
            no = self.node_to_int[curr_node]
            self.adj_list[no] = [map(lambda x: self.node_to_int[x], curr_node.in_adj), map(lambda x: self.node_to_int[x], curr_node.out_adj)]
#             for par in curr_node.in_adj:
#                 self.adj_mat[self.node_to_int[par]][no] = 1
#                 self.adj_list[self.node_to_int[par]][1].append(no)
            for child in curr_node.out_adj:
                self.adj_mat[no][self.node_to_int[child]] = 1
#                 self.adj_list[self.node_to_int[child]][0].append(no)
        self.topsorting()
    
    
    def remove():
        pass
    
    def deepen_morph(self):
        deepen_conv_block = self.int_to_node[random.choice(self.conv_blocks)]
        kernel_size = random.choice([3, 5])
        in_channels, in_h, in_w = deepen_conv_block.output_shape
        out_channels = in_channels
        identity_conv_block = convolution_block(in_channels, in_h, in_w, out_channels, kernel_size, (kernel_size-1)/2)
#         weights = identity_conv_block.conv_layer.weight.data
        weights = identity_conv_block.conv_layer.weight
        
        # creating identity weights
        for channel in range(out_channels):
            for i in range(in_channels):
                for j in range(kernel_size):
                    for k in range(kernel_size):
                        weights[channel][i][j][k] = int((channel == i) and (j == k) and j == (kernel_size)/2 )
        
        ## make connections 
        identity_conv_block.describe_adj_list([deepen_conv_block], deepen_conv_block.out_adj)
        deepen_conv_block.describe_adj_list(deepen_conv_block.in_adj, [identity_conv_block])

        #### later look at creating a function for singular change to in_adj or out_adj of nodes
        for out_node in identity_conv_block.out_adj:
            out_node_in_adj = [identity_conv_block if (x == deepen_conv_block) else x for x in out_node.in_adj ]
            out_node.describe_adj_list(out_node_in_adj, out_node.out_adj)
        
        self.add_nodes_to_network([deepen_conv_block, identity_conv_block] + identity_conv_block.out_adj)
        
    
    def widen_morph(self):
        candidate_conv_blocks = []
        for conv_block in self.conv_blocks:
            isCandidate = bool(len(self.adj_list[conv_block][1]))
            for child in self.adj_list[conv_block][1]:
                isCandidate = isCandidate and isinstance(self.int_to_node[child], convolution_block)
            if isCandidate:
                candidate_conv_blocks.append(conv_block)
        if len(candidate_conv_blocks) == 0:
            return False

#         parent_block_no = random.choice(candidate_conv_blocks)
        parent_block = self.int_to_node[random.choice(candidate_conv_blocks)]
        widening_factor = random.choice([2, 4])
        in_channels, in_h, in_w = parent_block.input_shape
        out_channels = parent_block.out_channels
        kernel_size = parent_block.kernel_size
        padding = parent_block.padding
        stride = parent_block.stride
#         original_parent_weight = parent_block.conv_layer.weight.data
        original_parent_weight = parent_block.conv_layer.weight
        #set_params
        parent_block.refresh(in_channels, in_h, in_w, out_channels*widening_factor, kernel_size, padding, stride)
        
#         parent_block.conv_layer = nn.Conv2d(in_channels, out_channels*widening_factor, kernel_size, stride, padding)
#         parent_block.batch_norm = nn.BatchNorm2d(out_channels*widening_factor)
#         widened_parent_block = convolution_block(in_h, in_w, in_channels, out_channels*widening_factor, kernel_size, padding, stride)
#         widened_parent_weight = parent_block.conv_layer.weight.data
        widened_parent_weight = parent_block.conv_layer.weight
#         print('New shape vs old shape', type(widened_parent_weight),widened_parent_weight.shape, original_parent_weight.shape)
        widened_parent_weight[:out_channels] =torch.nn.Parameter(original_parent_weight)
        widened_parent_weight[out_channels:] = torch.nn.Parameter(torch.zeros((out_channels*(widening_factor-1), in_channels, kernel_size, kernel_size)))

#         parent_out_adj = []
        for child in parent_block.out_adj:
            child_no = self.node_to_int[child]
            in_channels, in_h, in_w = child.input_shape
            out_channels = child.out_channels
            kernel_size = child.kernel_size 
            padding = child.padding
            stride = child.stride
            original_child_weight = child.conv_layer.weight
            child.refresh(in_channels*widening_factor, in_h, in_w, out_channels, kernel_size, padding, stride)
#             child.conv_layer = nn.Conv2d(in_channels*widening_factor, out_channels, kernel_size, stride, padding)
#             child.batch_norm = nn.BatchNorm2d(out_channels)
#             child_widened = convolution_block(in_h, in_w, in_channels*widening_factor, out_channels, kernel_size, padding, stride)
            child.conv_layer.weight[:, :in_channels, :, :] = torch.nn.Parameter(original_child_weight)
#             child_widened.describe_adj_list([widened_parent_block if x == parent_block else x for x in child.in_adj], child.out_adj)
#             child = child_widened

#         parent_block = widened_parent_block
        
    ## Start from here
    def dfs(self, curr_node, visited, weight):
        curr_node_obj = self.int_to_node[curr_node]
        if isinstance(curr_node_obj, FlattenBlock) or isinstance(curr_node_obj, fullyConnectedBlock):
            return
        visited[curr_node] = weight
        for child in self.adj_list[curr_node][1]:
            if child not in visited:
                kernel = 0
                padding = 0
                constant = 0
                child_node = self.int_to_node[child]
                ## make adjustments for concatenation
                if isinstance(child_node, convolution_block) or isinstance(child_node, max_pool_block):
                    kernel = child_node.kernel_size
                    padding = child_node.padding
                    constant = 1
                self.dfs(child, visited, [weight[0]+kernel, weight[1]+padding, weight[2]+constant])
        
        
    def get_descendant_vectors(self):
        descs = {}
        for curr_node in self.nodes:
            if isinstance(self.int_to_node[curr_node], FlattenBlock) or isinstance(self.int_to_node[curr_node], fullyConnectedBlock):
                continue
            visited = {}
            self.dfs(curr_node, visited, [0, 0, 0])
            del visited[curr_node] # remove root 
            descs[curr_node] = visited
        return descs
        
    def skip_morph(self):
        descs = self.get_descendant_vectors()
        candidates = [(ans, des) for ans in descs for des in descs[ans] ]
        no1, no2 = random.choice(candidates)
        weight = descs[no1][no2]
        #join outputs of node1 and node2 using a merge block
        node_a = self.int_to_node[no1]
        node_b = self.int_to_node[no2]
#         print('Skip Morphism', type(node_a), type(node_b))
        out_ch_1, out_h_1, out_w_1 = node_a.output_shape
        out_ch_2, out_h_2, out_w_2 = node_b.output_shape
#         print 'selected_nodes are ', no1, "  ",no2
#         print 'weight is   ', weight
#         print(out_h_1, out_h_2, weight[0] - 2*weight[1] - weight[2])
        assert(out_h_1 - out_h_2 == out_w_1 - out_w_2)
        assert(out_h_1 - out_h_2 == weight[0] - 2*weight[1] - weight[2])

        if weight[2] & 1 == 0:
            weight[2] += 1
            weight[0] += 1
        weight[1] += (weight[2])/2
        weight[2] -= 2*(weight[2]/2)
        kernel_size = weight[0]
        padding = weight[1]
        stride = weight[2]
        assert stride == 1
        new_conv = convolution_block(out_ch_1, out_h_1, out_w_1, out_ch_2, kernel_size, padding, stride)
        new_add = add_block([new_conv, node_b], node_b.out_adj)
        new_conv.describe_adj_list([node_a], [new_add])
        new_conv.conv_layer.weight = torch.nn.Parameter(torch.zeros(new_conv.conv_layer.weight.data.shape))
        node_a.describe_adj_list(node_a.in_adj, node_a.out_adj+[new_conv])
        for child_node in node_b.out_adj:
            child_node.describe_adj_list([new_add if x==node_b else x for x in child_node.in_adj], child_node.out_adj)
        node_b.describe_adj_list(node_b.in_adj, [new_add])
        self.add_nodes_to_network([node_a, node_b, new_conv, new_add] + new_add.out_adj)
        
        
    def visualize(self):
        graph = Digraph('./assets/images/test_net', './assets/images/test_net')
        for no, curr_node in self.int_to_node.items():
#             graph.node(str(no), str(type(curr_node)).split('__main__.')[1])
            graph.node(str(no), str(no) + " :: " + repr(curr_node))
        for no, li in self.adj_list.items():
            for ch in li[1]:
                graph.edge(str(no), str(ch))
        graph.view()

    
    def describe(self):
        print 'Nodes: ', self.nodes
        print 'Conv_blocks', self.conv_blocks
        print 'Max_pool_blocks', self.max_pool_blocks
        print 'Adj_list', self.adj_list
        print 'Adj_mat', self.adj_mat
        print 'int_to_node', self.int_to_node
        print 'node_to_int', self.node_to_int
        print 'Toposort', self.topsort
        print 'node_dict', self.node_dict
    

In [102]:
def test():    
#     dummy = node((3, 32, 32), (3, 32, 32))
    #     in_channels, in_h, in_w, out_channels, kernel_size, padding = 0, stride = 1
    n1 = convolution_block(3, 32, 32 , 4, 3)
    n2 = convolution_block(4, 30, 30, 3, 3)
    flatten = FlattenBlock((3, 28, 28))
    f1 = fullyConnectedBlock(2352, 64, True)
    f2 = fullyConnectedBlock(64, 32)
    
    n1.describe_adj_list([], [n2])
    n2.describe_adj_list([n1], [flatten])
    flatten.describe_adj_list([n2], [f1])
    f1.describe_adj_list([f1], [f2])
    f2.describe_adj_list([f1], [])
    

    net = Network({0:[[], [1]], 1:[[0], [2]], 2: [[1], [3]], 3:[[2], [4]], 4:[[3], []]}, {0: n1, 1:n2, 2:flatten, 3:f1, 4:f2})
#     net.describe() 
    net.visualize()
    for nan in net.nodes:
        print net.int_to_node[nan].input_shape, ' -> ', nan, '->', net.int_to_node[nan].output_shape 
    for i in range(20):
        operations = [net.skip_morph, net.deepen_morph, net.widen_morph]
        op = random.choice(operations)
        print "Iteration", i, op
        op()
    net.visualize()
#     net=addLinearLayers(net)
test()

(3, 32, 32)  ->  0 -> (4, 30, 30)
(4, 30, 30)  ->  1 -> (3, 28, 28)
(3, 28, 28)  ->  2 -> (2352,)
(2352,)  ->  3 -> (64,)
(64,)  ->  4 -> (32,)
Iteration 0 <bound method Network.widen_morph of Network()>
Iteration 1 <bound method Network.skip_morph of Network()>
Iteration 2 <bound method Network.skip_morph of Network()>
Iteration 3 <bound method Network.deepen_morph of Network()>
Iteration 4 <bound method Network.deepen_morph of Network()>
Iteration 5 <bound method Network.deepen_morph of Network()>
Iteration 6 <bound method Network.widen_morph of Network()>
Iteration 7 <bound method Network.deepen_morph of Network()>
Iteration 8 <bound method Network.widen_morph of Network()>
Iteration 9 <bound method Network.widen_morph of Network()>
Iteration 10 <bound method Network.skip_morph of Network()>
Iteration 11 <bound method Network.deepen_morph of Network()>
Iteration 12 <bound method Network.widen_morph of Network()>
Iteration 13 <bound method Network.widen_morph of Network()>
Iteration 

In [93]:
! ls ./assets/images

test_net.gv     test_net.gv.pdf
